In [1]:
import ctypes
from ctypes import *
import numpy as np

## write wrapper for Gauss2D_analysis

In [321]:
%%file Gauss2D_dummy.cpp

#include <stdio.h>

// fitting options
typedef struct {
    int threshold; // threshold for peak search
    char relative_thr;		// threshold is relative to bg_surface
    int offset;			// offset (max +/- deltax, deltay) for fitting
    int maxNPeaks;
    char fit2DGauss;		// fit or just return cm's
    double sigma0;			// initial estimation for sigma
    double dsigma_max;		// threshold for |sigma_x-sigma_y|/(sigma_x+sigma-y)
    char must_converge;		// discard peaks for which LM has not converged
    int max_threshold; 			// max_threshold for peak search
} OptsCluster;

// fit results
typedef struct {
    int imageID;
    } ResultsCluster;

extern "C" int Gauss2D(int * image,
            int size_x,
            int size_y,
            OptsCluster * options,
            int Nimage,
            int& Nall,
            int& Ngood,
            int& Nconverged,
            int& Npeaks,
            void** presults){
    int i, j;
    for(i = 0; i < size_x * size_y; i++){
        printf("pixel value %d is %d\n", i, image[i]);
    }
    printf("size x value is %d and size y value is %d \n", size_x, size_y);
    printf("options threshold is %d \n", options->threshold);
    printf("option relative threshold is %c \n", options->relative_thr);
    printf("option sigma0 is %f \n", options->sigma0);
    printf("option max threshold is %d \n", options->max_threshold);
    printf("Nimage is %d\n", Nimage);
    printf("Npeaks is %d\n", Npeaks);
        
    // first 4 bytes contain the array size -- must skip. See LabView help.
    ResultsCluster* results = (ResultsCluster*)((__int64*)(*presults) + 1);
    printf("presults is %x\n", *presults);
    
    for( i = 0; i < 3; i++) {
        results[i].imageID = i;
    }
    /*
    for (i =0; i<3; i++){
        printf("element %d of results is %d\n", i, results[i].imageID);
    }
    printf("results is %x\n", *results);

    */
    return 0;
}

Overwriting Gauss2D_dummy.cpp


In [322]:
!gcc -c -o dummy.o Gauss2D_dummy.cpp
!gcc -o dummy.dll -shared dummy.o

In [323]:
%%file dummy_wrapper.py

import numpy as np
from ctypes import *

class Options(Structure):
    _fields_ = [
        ("threshold", c_int),
        ("relative_thr", c_char),
        ("offset", c_int),
        ("maxNpeaks", c_int),
        ("fit2DGauss", c_char),
        ("sigma0",c_double),
        ("dsigma0_max", c_double),
        ("must_converge", c_char),
        ("max_threshold", c_int)
    ]


_Gauss2D = CDLL('dummy.dll').Gauss2D
_Gauss2D.argtypes = [np.ctypeslib.ndpointer(dtype=np.int), 
                     c_int, 
                     c_int,
                     POINTER(Options),
                     c_int,
                     np.ctypeslib.ndpointer(dtype=np.int),
                     np.ctypeslib.ndpointer(dtype=np.int),
                     np.ctypeslib.ndpointer(dtype=np.int),
                     np.ctypeslib.ndpointer(dtype=np.int),
                     POINTER(c_void_p)]
_Gauss2D.restype = c_int



def Gauss2D(image, options, Nimage = 1, Nall = 1, Ngood = 1, Nconverged = 1, Npeaks = 1):
    size_x, size_y = image.shape
    image = np.asarray(image.ravel(), dtype=np.int)
    Nall = np.asarray(Nall, dtype=np.int)
    Ngood = np.asarray(Ngood, dtype=np.int)
    Nconverged = np.asarray(Nconverged, dtype=np.int)
    Npeaks = np.asarray(Npeaks, dtype=np.int)
    presults = c_void_p(0)
    _Gauss2D(image, size_x, size_y, options, Nimage, Nall, Ngood, Nconverged, Npeaks, byref(presults))
    return presults

Overwriting dummy_wrapper.py


In [324]:
%%file run_Gauss2D_dummy.py

import dummy_wrapper as wrapper
import numpy as np

image = np.linspace(0,20,15).reshape(3,5)
options = wrapper.Options(1,b'0',1,1,b'0',2.5,5.0,0, 3)
presults = wrapper.Gauss2D(image, options)
print("presult value in pyhon is " + str(presults.value))

Overwriting run_Gauss2D_dummy.py


In [325]:
!python run_Gauss2D_dummy.py

pixel value 0 is 0
pixel value 1 is 1
pixel value 2 is 2
pixel value 3 is 4
pixel value 4 is 5
pixel value 5 is 7
pixel value 6 is 8
pixel value 7 is 10
pixel value 8 is 11
pixel value 9 is 12
pixel value 10 is 14
pixel value 11 is 15
pixel value 12 is 17
pixel value 13 is 18
pixel value 14 is 20
size x value is 3 and size y value is 5 
options threshold is 1 
option relative threshold is 0 
option sigma0 is 2.500000 
option max threshold is 3 
Nimage is 1
Npeaks is 1
presults is 0


Traceback (most recent call last):
  File "run_Gauss2D_dummy.py", line 7, in <module>
    presults = wrapper.Gauss2D(image, options)
  File "K:\vanderVoortN\FRC\python\Code\dummy_wrapper.py", line 42, in Gauss2D
    _Gauss2D(image, size_x, size_y, options, Nimage, Nall, Ngood, Nconverged, Npeaks, byref(presults))
OSError: exception: access violation writing 0x0000000000000008


In [269]:
%%file understanding_pointers.cpp

#include <stdio.h> 
#include <stdlib.h> 

int main(){
    
    printf("hello world!\n");
    
    void **array;
    int arrayLen = 10;
    
    array = (void**)malloc(arrayLen * sizeof(void*));
    
    int * intp, i;
    for (i = 0; i >10; i++) {
        array[i] = (void*) intp;
    }

    for ( i= 0; i < 10; i++) {
        printf("element %d of array is %x \n",i, array[i]);
    }
    return 1;
}

Overwriting understanding_pointers.cpp


In [268]:
!gcc -o understanding_pointers.exe understanding_pointers.cpp
!understanding_pointers.exe

understanding_pointers.cpp: In function 'int main()':
understanding_pointers.cpp:20:58: error: 'void*' is not a pointer-to-object type
         printf("element %d of array is %x \n",i, *array[i]);
                                                          ^


hello world!
element 0 of array is 777490 
element 1 of array is 770150 
element 2 of array is 4d4f5250 
element 3 of array is 4649444f 
element 4 of array is 73616228 
element 5 of array is 444e4f43 
element 6 of array is 4e4f4854 
element 7 of array is 5c3a433d 
element 8 of array is 446d6172 
element 9 of array is 63616e41 


## Write wrapper for test functions

In [56]:
%%file functions.c

#include <stdio.h>

typedef struct Options{
    char mode[50];
}Option;

void hello(Option option);

double dprod(double *x, int n);

void dcumsum(double *a, double *b, int n);

void
hello(Option option)
{
    printf("C says hello\n");
    printf("mode is %s\n", option.mode);
}


double 
dprod(double *x, int n)
{
    int i;
    double y = 1.0;
    
    for (i = 0; i < n; i++)
    {
        y *= x[i];
    }

    return y;
}

void
dcumsum(double *a, double *b, int n)
{
    int i;
    
    b[0] = a[0];
    for (i = 1; i < n; i++)
    {
        b[i] = a[i] + b[i-1];
    }
}

#define SMB_MAX_DATA_SIZE 5
typedef void* SMBUS_HANDLE;

typedef struct _SMB_REQUEST
{
    unsigned char Address;
    unsigned char Command;
    unsigned char BlockLength;
    unsigned char Data[SMB_MAX_DATA_SIZE];
} SMB_REQUEST;

int SmBusReadByte(SMBUS_HANDLE handle,SMB_REQUEST *request)
{
    unsigned char i;
    for(i = 0; i < request->BlockLength; i++)
        request->Data[i] = i;
    return request->BlockLength;
}

SMBUS_HANDLE OpenSmbus(void)
{
    return (void*)0x12345678;
}


Overwriting functions.c


In [57]:
#!gcc -c -Wall -O2 -Wall -ansi -pedantic -fPIC -o functions.o functions.c
!gcc -c -o functions.o functions.c
!gcc -o libfunctions.dll -shared functions.o

In [58]:
%%file functions.py

import numpy
import ctypes
from ctypes import *

class Options(Structure):
    _fields_ = [
        ("mode", c_char)
    ]

_libfunctions = numpy.ctypeslib.load_library('libfunctions', '.')

_libfunctions.hello.argtypes = [POINTER(Options)]
_libfunctions.hello.restype  =  ctypes.c_void_p

_libfunctions.dprod.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), ctypes.c_int]
_libfunctions.dprod.restype  = ctypes.c_double

_libfunctions.dcumsum.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), numpy.ctypeslib.ndpointer(dtype=numpy.float), ctypes.c_int]
_libfunctions.dcumsum.restype  = ctypes.c_void_p

def hello():
    option = Options(b"r")
    return _libfunctions.hello(option)

def dprod(x, n=None):
    if n is None:
        n = len(x)
    x = numpy.asarray(x, dtype=numpy.float)
    return _libfunctions.dprod(x, int(n))

def dcumsum(a, n):
    a = numpy.asarray(a, dtype=numpy.float)
    b = numpy.empty(len(a), dtype=numpy.float)
    _libfunctions.dcumsum(a, b, int(n))
    return b

Overwriting functions.py


In [22]:
%%file run_hello_c.py

import functions

functions.hello()

Overwriting run_hello_c.py


In [17]:
%%file run_struct_func_c

from ctypes import *
SMB_MAX_DATA_SIZE = 5
ARRAY5 = c_ubyte * SMB_MAX_DATA_SIZE

class SMB_REQUEST(Structure):
    _fields_ = [
        ("Address", c_ubyte),
        ("Command", c_ubyte),
        ("BlockLength", c_ubyte),
        ("Data", ARRAY5)]

smbus_read_byte = CDLL('libfunctions').SmBusReadByte
smbus_read_byte.argtypes = [c_void_p,POINTER(SMB_REQUEST)]
smbus_read_byte.restype = c_int
open_smbus = CDLL('libfunctions').OpenSmbus
open_smbus.argtypes = []
open_smbus.restype = c_void_p

handle = open_smbus()
print ('handle = %08Xh' % handle)

smb_request = SMB_REQUEST(1,2,5)

print ('returned =',smbus_read_byte(handle,byref(smb_request)))
print ('Address =',smb_request.Address)
print ('Command =',smb_request.Command)
print ('BlockLength =',smb_request.BlockLength)
for i,b in enumerate(smb_request.Data):
    print ('Data[%d] = %02Xh' % (i,b))

Overwriting run_struct_func_c


In [37]:
!python run_struct_func_c

handle = 12345678h
returned = 5
Address = 1
Command = 2
BlockLength = 5
Data[0] = 00h
Data[1] = 01h
Data[2] = 02h
Data[3] = 03h
Data[4] = 04h


In [36]:
!python run_hello_c.py

C says hello
mode is r


In [ ]:
import functions
x = [1,2,4,5]
functions.dprod(x)

In [1]:
2**16

65536

## wrappers using pybind and cppimport

In [1]:
import sys
import cppimport

In [2]:
sys.path.append(r'K:\vanderVoortN\FRC\dev\sandbox\python_cpp_example\python_cpp_example\Debug')#export location of cfiles
sys.path.append(r'K:\vanderVoortN\FRC\dev\sandbox\python_cpp_example\src\python_cpp_example')#lib or cppimport

In [10]:
import python_cpp_example as pce
cppimport.set_quiet(False)

In [4]:
pce.inv

<function python_cpp_example.PyCapsule.inv>

In [5]:
funcs = cppimport.imp("bindings")

SystemExit: error: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\Community\\VC\\Tools\\MSVC\\14.16.27023\\bin\\HostX86\\x64\\cl.exe' failed with exit status 2

In [6]:
%tb

SystemExit: error: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2017\\Community\\VC\\Tools\\MSVC\\14.16.27023\\bin\\HostX86\\x64\\cl.exe' failed with exit status 2

In [8]:
if 1: print("hello world!")

hello world!
